<a href="https://colab.research.google.com/github/buganart/BUGAN/blob/master/notebook_util/download_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Before starting please save the notebook in your drive by clicking on `File -> Save a copy in drive`

In [ ]:
#@markdown Mount google drive.
from google.colab import output
from google.colab import drive
drive.mount('/content/drive')

# Check if we have linked the folder
from pathlib import Path
if not Path("/content/drive/My Drive/IRCMS_GAN_collaborative_database").exists():
    print(
        "Shortcut to our shared drive folder doesn't exits.\n\n"
        "\t1. Go to the google drive web UI\n"
        "\t2. Right click shared folder IRCMS_GAN_collaborative_database and click \"Add shortcut to Drive\""
    )

In [ ]:
#@markdown Install wandb and log in
%pip install wandb
output.clear()
import wandb
wandb_drive_netrc_path = Path("drive/My Drive/colab/.netrc")
wandb_local_netrc_path = Path("/root/.netrc")
if wandb_drive_netrc_path.exists():
    import shutil

    print("Wandb .netrc file found, will use that to log in.")
    shutil.copy(wandb_drive_netrc_path, wandb_local_netrc_path)
else:
    print(
        f"Wandb config not found at {wandb_drive_netrc_path}.\n"
        f"Using manual login.\n\n"
        f"To use auto login in the future, finish the manual login first and then run:\n\n"
        f"\t!mkdir -p '{wandb_drive_netrc_path.parent}'\n"
        f"\t!cp {wandb_local_netrc_path} '{wandb_drive_netrc_path}'\n\n"
        f"Then that file will be used to login next time.\n"
    )

!wandb login
output.clear()
print("ok!")

In [ ]:
#@title Configure dataset

#@markdown Enter run information. 
#@markdown - set `"run_id"` to resume a run (for example: `u9imsvva`)
#@markdown - The id of the current run is shown below in the cell with `wandb.init()` or you can find it in the wandb web UI.
id = "3tiaj8j1" #@param {type:"string"}
#@markdown Enter project name (either `chair-gan`, `handtool-gan` or `tree-gan`)
project_name = "handtool-gan" #@param ["tree-gan", "handtool-gan", "chair-gan"]

#@markdown Enter export location (folder/directory).
#@markdown - If leave blank, the file will be saved to colab '/tmp/' folder   
export_location = f"/content/drive/MyDrive/IRCMS_GAN_collaborative_database/Experiments/RunFileDirectory/{id}" #@param {type:"string"}

#@markdown If True, the files saved to colab drive will be zipped and downloaded using browser. 
download_zip = True    #@param {type:"boolean"}

if (not id) or (not project_name):
    raise Exception("Please set id and project_name!")
# export_location = str(export_location)
if not export_location:
    export_location = "/tmp/"

print("id:", id)
print("project_name:", project_name)
print("export_location:", export_location)
print("download_zip:", download_zip)

In [ ]:
import os
from argparse import Namespace, ArgumentParser
from pathlib import Path
#@markdown download files
os.environ["WANDB_MODE"] = "dryrun"

export_location = Path(export_location)
if not export_location.exists():
    export_location.mkdir(parents=True)
%cd {export_location}


entity="bugan"
rev_number = None

#read information (run config, etc) stored online
    #all config will be replaced by the stored one in wandb
api = wandb.Api()
previous_run = api.run(f"{entity}/{project_name}/{id}")
print("start download from wandb!")
for file in previous_run.files():
    file.download(replace=True)
print("download complete!")

#zip files in the directory
if download_zip:
    !zip -r /content/file.zip .
    from google.colab import files
    files.download("/content/file.zip")

print("ok!")